#### ANALYSE Alternative 1: Vereinfachtes Vorgehen durch Aggregation (Mittelwert)

Liefert identische Ergebnisse zu Alternative 2, sofern jeder der vier Sensoren je pro ID einmal in Daten vorkommt. In vorliegenden Fall gegeben.

In [1]:
df_mean = (
    df
    .groupby("id")[variables]  # nur die Original-Spalten auswählen
    .mean()  # über id mitteln
    .add_suffix("_mean")  # hier hängen wir das Suffix an
    .reset_index()
    .merge(
        df[add_variables].drop_duplicates(),
        on="id"
    )
)

df_mean.describe()

NameError: name 'df' is not defined

#### ANALYSE Alternative 2: Werte um Sensor-Effekt und Vorspannung zu adjustieren und dann zu aggregieren (Mittelwert)


In [ ]:
model_dict = {}
# Erstelle eine Kopie des Original-DataFrames
df_copy = df.copy()

global_means = {var: df[var].mean() for var in variables}

for var in variables:
    try:
        # Sum-Kontraste: Intercept = globaler Mittelwert
        model = smf.ols(f"{var} ~ C(sensor_name, Sum)", data=df).fit()
        model_dict[var] = model

        # resid = y - (Intercept + Sensor-Abweichung)
        # und resid + Intercept gibt y ohne Sensor-Effekt, zentriert am globalen Mittelwert
        df_copy[f"{var}_adj"] = model.resid + model.params['Intercept']
    except Exception as e:
        print(f"Fehler bei {var}: {e}")

# Aggregiere über die ID – hier wird der Mittelwert der adjustierten Werte berechnet.
df_adj = (
    df_copy
    .groupby("id")
    .agg({f"{var}_adj": "mean" for var in variables})
    .reset_index()
    .merge(df[add_variables].drop_duplicates(), on="id")
)

df_adj.describe()

Folgende Prüfung sollte nahe Null ergeben, wenn Skalierung und mittelwerte erhalten geblieben sind:

In [ ]:
# Erstelle Listen mit den aktuellen Spaltennamen
raw_cols = [f"{var}_mean" for var in variables]
adj_cols = [f"{var}_adj" for var in variables]

# Berechne die ungewichteten ID-Mittelwerte
mean_id_raw = df_mean[raw_cols].mean()
mean_id_adj = df_adj[adj_cols].mean()

# Die Series haben jetzt noch die Suffix-Namen – indexe sie auf die Original-Variable
mean_id_raw.index = variables
mean_id_adj.index = variables

# Differenz ausgeben
diff = mean_id_raw - mean_id_adj
print(diff)

##### LATEX-EXPORT: Modell-Info

In [ ]:
def extract_model_info(model):
    """
    Extrahiert zentrale Kennzahlen aus einem OLS-Modell.
    Liefert ein Dictionary mit R-squared, Adj. R-squared, sowie den Koeffizienten,
    Standardfehler, t-Werten und p-Werten für alle Parameter.
    """
    info = {
        "R²": model.rsquared,
        "adj_R²": model.rsquared_adj,
    }
    # Iteriere über alle Parameter im Modell
    for param in model.params.index:
        info[f"coef_{param}"] = model.params[param]
        info[f"std_err_{param}"] = model.bse[param]
        info[f"t_{param}"] = model.tvalues[param]
        info[f"p_{param}"] = model.pvalues[param]
    return info


# Erstelle eine Liste, in der pro Variable das extrahierte Dictionary gespeichert wird.
model_info_list = []
for var, model in model_dict.items():
    info = extract_model_info(model)
    info['variable'] = var  # Variable als Kennung hinzufügen
    model_info_list.append(info)

# Konvertiere die Liste in einen DataFrame und setze "variable" als Index
df_model_info = pd.DataFrame(model_info_list).set_index("variable")
df_model_info.index.name = "Statistik"
df_model_info = df_model_info.T

In [ ]:
def format_value(x):
    # Prüfe, ob der Wert numerisch ist
    if isinstance(x, (int, float, np.number)):
        if abs(x) < 0.0001:
            return f"{x:.2e}"
        else:
            return f"{x:.2f}"
    return x


# Wende die Funktion auf alle Zellen von df_model_info an
df_model_info = df_model_info.applymap(format_value)
df_model_info

In [ ]:
# Manuelles Escapen der Unterstriche im Index
df_model_info.index = df_model_info.index.astype(str).str.replace('(sensor_name)[T.', r'(sensor)[')
df_model_info.index = df_model_info.index.astype(str).str.replace('_', r'\_')

In [ ]:
# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_model_info = df_model_info.rename(columns={var: data_dict[var]["Zeichen"] for var in variables})

column_format = "l" + "r" * (len(df_model_info.columns))
latex_string = df_model_info.to_latex(index=True, escape=False, float_format="%.2f", column_format=column_format)

# Definiere Beschriftung und lange Beschriftung (Caption)
caption = "Feldversuch 2: Ergebnisse, Zusammenfassung der OLS-Modelle"
caption_long = "Übersicht über zentrale Kennzahlen der OLS-Modelle: R², Adjusted R², Koeffizienten, Standardfehler, t-Werte und p-Werte für alle Parameter."

# Funktion zum Speichern der LaTeX-Tabelle (du passt diese Funktion ggf. an deine Gegebenheiten an)
save_latex_table(latex_string, caption, latex_export_directory, caption_long)

#### Zusammenführen beider Alternativen für Vergleich

In [ ]:

# --- Zusammenführen beider Ansätze ---
# Beide DataFrames enthalten nun jeweils den Mittelwert pro ID:
# - df_mean: direkte Mittelwerte mit Suffix _mean
# - df_adj: sensor-adjustierte Mittelwerte mit Suffix _adj
grouped_df = pd.merge(df_mean, df_adj, on=add_variables, how="inner")
grouped_df

#### VISUALISIERUNG: Stelle beide Methoden der Aggregation gegenüber

In [ ]:
for i, var in enumerate(variables):
    # Transformiere die Daten in das Long-Format: 
    # Es werden die beiden Spalten f"{var}_mean" und f"{var}_adj" unter einer neuen Spalte "aggregation" zusammengeführt,
    # und die Werte in der Spalte "value" gespeichert.
    df_long = pd.melt(
        grouped_df,
        id_vars=["id", "treatment", "rope_release"],
        value_vars=[f"{var}_mean", f"{var}_adj"],
        var_name="aggregation",
        value_name="value"
    )
    # Kürze die Beschriftungen, sodass nur "mean" bzw. "adj" übrig bleiben
    df_long["aggregation"] = df_long["aggregation"].str.replace(f"{var}_", "", regex=False)

    # Erstelle den Plot: Boxplots nach Behandlung, getrennt nach Aggregationsmethode
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="treatment", y="value", hue="aggregation", data=df_long, dodge=True)

    # Titel und Achsentitel setzen
    plt.title(f"Einfluss der Behandlung auf {get_label_from_dict(var, data_dict, use_titel=True)}")
    plt.xlabel("Behandlung")
    plt.ylabel(get_label_from_dict(var, data_dict, use_full=True))
    plt.tight_layout()

    # Speichere den Plot (ersetze PLOT_MANAGER.save_plot durch plt.show() oder deine eigene Funktion, falls nötig)
    PLOT_MANAGER.save_plot(fig, filename=f"effect_treatment_{i}_{var}",
                           subdir="ptq_osc_sensor_effect_4_comparision_mean_vs_adj")

#### SET: Übernimm um Sensorposition adjustierte Werte

Die statistische Bereinigung um den Effekt der Sensoren erscheint geeigneter, die daten werden nachfolgend weiter verwendet

In [ ]:
# 1. Alle "_adj"‑Suffixe aus den Spaltennamen entfernen
df_adj = df_adj.rename(columns=lambda x: x[:-4] if x.endswith("_adj") else x)

# 2. Gewünschte Spalten vorne in der Reihenfolge id, treatment, rope_release, cable_max
front = add_variables
# Alle übrigen Spalten, in der vorhandenen Reihenfolge
rest = [c for c in df_adj.columns if c not in front]

# 3. DataFrame umsortieren
df_adj = df_adj[front + rest]
df_adj